<a href="https://colab.research.google.com/github/Swastik200/30DaysOfDSA/blob/main/Genetic_Algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyMetaheuristic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.4/98.4 kB 1.5 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.linear_model import LinearRegression
from pyMetaheuristic.algorithm import genetic_algorithm
from pyMetaheuristic.utils import graphs

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!cp "/content/drive/MyDrive/CMAPSS.zip" "/content"
!unzip CMAPSS.zip

Archive:  CMAPSS.zip
  inflating: CMAPSS/readme.txt       
  inflating: CMAPSS/RUL_FD001.txt    
  inflating: CMAPSS/RUL_FD002.txt    
  inflating: CMAPSS/RUL_FD003.txt    
  inflating: CMAPSS/RUL_FD004.txt    
  inflating: CMAPSS/test_FD001.txt   
  inflating: CMAPSS/test_FD002.txt   
  inflating: CMAPSS/test_FD003.txt   
  inflating: CMAPSS/test_FD004.txt   
  inflating: CMAPSS/train_FD001.txt  
  inflating: CMAPSS/train_FD002.txt  
  inflating: CMAPSS/train_FD003.txt  
  inflating: CMAPSS/train_FD004.txt  
  inflating: CMAPSS/x.txt            


In [5]:
index_names = ['unit_number', 'time_cycles']
setting_names = ['setting_1', 'setting_2', 'setting_3']
sensor_names = ['sensor_{}'.format(i) for i in range(1, 22)]
col_names = index_names + setting_names + sensor_names

In [6]:
path = '/content/CMAPSS/'
df_train = pd.read_csv(path + 'train_FD001.txt', sep='\s+', header=None, index_col=False, names=col_names)
df_test = pd.read_csv(path + 'test_FD001.txt', sep='\s+', header=None, index_col=False, names=col_names)
y_test = pd.read_csv(path + 'RUL_FD001.txt', sep='\s+', header=None, index_col=False, names=['RUL'])

In [8]:
# Calculate 'RUL' column based on 'time_cycles' and 'unit_number' columns
max_cycles = df_train.groupby('unit_number')['time_cycles'].max()
df_train['RUL'] = df_train.apply(lambda row: max_cycles[row['unit_number']] - row['time_cycles'], axis=1)
# Calculate 'RUL' column based on 'time_cycles' and 'unit_number' columns
max_cycles = df_test.groupby('unit_number')['time_cycles'].max()
df_test['RUL'] = df_test.apply(lambda row: max_cycles[row['unit_number']] - row['time_cycles'], axis=1)


In [19]:
train = df_train.copy()
test = df_test.copy()
display(train)
display(test)

,unit_number,time_cycles,setting_1,setting_2,setting_3,sensor_1,sensor_2,sensor_3,sensor_4,sensor_5,...,sensor_13,sensor_14,sensor_15,sensor_16,sensor_17,sensor_18,sensor_19,sensor_20,sensor_21,RUL
0,1,1,-0.0007,-0.0004,100.0,518.67,641.82,1589.70,1400.60,14.62,...,2388.02,8138.62,8.4195,0.03,392,2388,100.0,39.06,23.4190,191.0
1,1,2,0.0019,-0.0003,100.0,518.67,642.15,1591.82,1403.14,14.62,...,2388.07,8131.49,8.4318,0.03,392,2388,100.0,39.00,23.4236,190.0
2,1,3,-0.0043,0.0003,100.0,518.67,642.35,1587.99,1404.20,14.62,...,2388.03,8133.23,8.4178,0.03,390,2388,100.0,38.95,23.3442,189.0
3,1,4,0.0007,0.0000,100.0,518.67,642.35,1582.79,1401.87,14.62,...,2388.08,8133.83,8.3682,0.03,392,2388,100.0,38.88,23.3739,188.0
4,1,5,-0.0019,-0.0002,100.0,518.67,642.37,1582.85,1406.22,14.62,...,2388.04,8133.80,8.4294,0.03,393,2388,100.0,38.90,23.4044,187.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20626,100,196,-0.0004,-0.0003,100.0,518.67,643.49,1597.98,1428.63,14.62,...,2388.26,8137.60,8.4956,0.03,397,2388,100.0,38.49,22.9735,4.0
20627,100,197,-0.0016,-0.0005,100.0,518.67,643.54,1604.50,1433.58,14.62,...,2388.22,8136.50,8.5139,0.03,395,2388,100.0,38.30,23.1594,3.0
20628,100,198,0.0004,0.0000,100.0,518.67,643.42,1602.46,1428.18,14.62,...,2388.24,8141.05,8.5646,0.03,398,2388,100.0,38.44,22.9333,2.0
20629,100,199,-0.0011,0.0003,100.0,518.67,643.23,1605.26,1426.53,14.62,...,2388.23,8139.29,8.5389,0.03,395,2388,100.0,38.29,23.0640,1.0


,unit_number,time_cycles,setting_1,setting_2,setting_3,sensor_1,sensor_2,sensor_3,sensor_4,sensor_5,...,sensor_13,sensor_14,sensor_15,sensor_16,sensor_17,sensor_18,sensor_19,sensor_20,sensor_21,RUL
0,1,1,0.0023,0.0003,100.0,518.67,643.02,1585.29,1398.21,14.62,...,2388.03,8125.55,8.4052,0.03,392,2388,100.0,38.86,23.3735,30.0
1,1,2,-0.0027,-0.0003,100.0,518.67,641.71,1588.45,1395.42,14.62,...,2388.06,8139.62,8.3803,0.03,393,2388,100.0,39.02,23.3916,29.0
2,1,3,0.0003,0.0001,100.0,518.67,642.46,1586.94,1401.34,14.62,...,2388.03,8130.10,8.4441,0.03,393,2388,100.0,39.08,23.4166,28.0
3,1,4,0.0042,0.0000,100.0,518.67,642.44,1584.12,1406.42,14.62,...,2388.05,8132.90,8.3917,0.03,391,2388,100.0,39.00,23.3737,27.0
4,1,5,0.0014,0.0000,100.0,518.67,642.51,1587.19,1401.92,14.62,...,2388.03,8129.54,8.4031,0.03,390,2388,100.0,38.99,23.4130,26.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13091,100,194,0.0049,0.0000,100.0,518.67,643.24,1599.45,1415.79,14.62,...,2388.00,8213.28,8.4715,0.03,394,2388,100.0,38.65,23.1974,4.0
13092,100,195,-0.0011,-0.0001,100.0,518.67,643.22,1595.69,1422.05,14.62,...,2388.09,8210.85,8.4512,0.03,395,2388,100.0,38.57,23.2771,3.0
13093,100,196,-0.0006,-0.0003,100.0,518.67,643.44,1593.15,1406.82,14.62,...,2388.04,8217.24,8.4569,0.03,395,2388,100.0,38.62,23.2051,2.0
13094,100,197,-0.0038,0.0001,100.0,518.67,643.26,1594.99,1419.36,14.62,...,2388.08,8220.48,8.4711,0.03,395,2388,100.0,38.66,23.2699,1.0


In [9]:
X_train = df_train.drop(columns=['RUL'])
y_train = df_train['RUL']
X_test = df_test.drop(columns=['RUL'])
y_test = y_test['RUL']


In [10]:
parameters = {
    'population_size': 250,
    'min_values': [-5] * len(X_train.columns),  # Adjust according to your feature space
    'max_values': [5] * len(X_train.columns),  # Adjust according to your feature space
    'generations': 10,
    'mutation_rate': 0.1,
    'elite': 1,
    'eta': 1,
    'mu': 1,
    'verbose': True,
    'start_init': None,
    'target_value': None
}

In [11]:
def target_function(variables_values):
    # Train the regression model
    model = LinearRegression()
    model.fit(X_train, y_train)
    # Predict using the trained model
    predictions = model.predict(X_test)
    # Calculate RMSE
    rmse = np.sqrt(mean_squared_error(y_test, predictions))
    return rmse

In [14]:
# Assuming you have already loaded your dataset into variables df_train and df_test

# Define X and y for the training dataset
X_train = df_train.drop(columns=['RUL'])  # Features
y_train = df_train['RUL']  # Target variable

# Define X and y for the testing dataset
X_test = df_test.drop(columns=['RUL'])  # Features
y_test = df_test['RUL']  # Target variable

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=42)


In [15]:
ga_result = genetic_algorithm(target_function=target_function, **parameters)

Generation =  0  f(x) =  39.22801459451258
Generation =  1  f(x) =  39.22801459451258
Generation =  2  f(x) =  39.22801459451258
Generation =  3  f(x) =  39.22801459451258
Generation =  4  f(x) =  39.22801459451258
Generation =  5  f(x) =  39.22801459451258
Generation =  6  f(x) =  39.22801459451258
Generation =  7  f(x) =  39.22801459451258
Generation =  8  f(x) =  39.22801459451258
Generation =  9  f(x) =  39.22801459451258
Generation =  10  f(x) =  39.22801459451258


In [16]:
optimized_model = LinearRegression().fit(X_test, y_test)

In [17]:
rmse_test = np.sqrt(mean_squared_error(y_test, optimized_model.predict(X_test)))
r2_test = r2_score(y_test, optimized_model.predict(X_test))
mae_test = mean_absolute_error(y_test, optimized_model.predict(X_test))

In [18]:
print('RMSE (Test):', rmse_test)
print('R2 Score (Test):', r2_test)
print('MAE (Test):', mae_test)

RMSE (Test): 38.96712711655703
R2 Score (Test): 0.6676504517530486
MAE (Test): 30.025205313555507
